In [ ]:
import os
import xml.etree.ElementTree as ET
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

class RSSContentExtractor:
    def __init__(self, rss_file_path, output_file='output.txt', max_workers=5):
        self.rss_file_path = rss_file_path
        self.output_file = output_file
        self.max_workers = max_workers

    def load_rss_file(self):
        if not os.path.exists(self.rss_file_path):
            logging.error(f"RSS file not found: {self.rss_file_path}")
            return []

        try:
            tree = ET.parse(self.rss_file_path)
            root = tree.getroot()

            # Handles different RSS XML structures
            links = root.findall('.//link')

            if not links:
                logging.warning("No links found in RSS file")
                return []

            return [link.text for link in links if link.text]

        except ET.ParseError:
            logging.error("Invalid XML structure")
            return []

    def fetch_content(self, url):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            logging.error(f"Error fetching {url}: {e}")
            return None

    def extract_contents(self):
        links = self.load_rss_file()

        if not links:
            logging.error("No links available to process")
            return

        with open(self.output_file, 'w', encoding='utf-8') as outfile:
            with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                future_to_url = {executor.submit(self.fetch_content, link): link for link in links}

                for future in as_completed(future_to_url):
                    url = future_to_url[future]
                    try:
                        content = future.result()
                        if content:
                            outfile.write(f"URL: {url}\n{content}\n{'='*50}\n")
                    except Exception as e:
                        logging.error(f"Error processing {url}: {e}")

def main():
    rss_file = 'feed.xml'  # Replace with your RSS XML file path
    extractor = RSSContentExtractor(rss_file)
    extractor.extract_contents()

if __name__ == "__main__":
    main()